In [1]:
import numpy as np

from pydrake.common.value import Value
from pydrake.geometry import StartMeshcat, MeshcatVisualizer
from pydrake.systems.framework import DiagramBuilder, LeafSystem, BasicVector
from pydrake.multibody.math import SpatialForce
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph, ExternallyAppliedSpatialForce, ExternallyAppliedSpatialForceMultiplexer
from pydrake.multibody.parsing import Parser
from pydrake.all import Simulator


# pydot = interface to graphViz,consist in a parser for the DotFormat used by GraphViz
from pydot import graph_from_dot_data

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
class Controller(LeafSystem):

    def __init__(self): 
        LeafSystem.__init__(self)
        #self.DeclareVectorInputPort("u", BasicVector(2)) 
        self.DeclareVectorOutputPort("y", BasicVector(2), self.CalcOutputY)
        self.t = 50
    def CalcOutputY(self, context, output): 
        q1dotdot = 0
        q2dotdot = 0
        output.SetFromVector([q1dotdot, q2dotdot])
    

class CartesianForceEE_Sys(LeafSystem):
    def __init__(self, end_effector_id):
        LeafSystem.__init__(self)
        self.force_obj = ExternallyAppliedSpatialForce()
        self.force_obj.body_index = end_effector_id
        force_cls = Value[list[ExternallyAppliedSpatialForce]]
        self.DeclareAbstractOutputPort("F-cartesian", 
                                       lambda: force_cls(), 
                                       self.CalcSpatialForce)

    def CalcSpatialForce(self, context, applied_force):
        F = SpatialForce(tau=np.zeros(3), f=np.array([1., 0., 0.]))
        self.force_obj.F_Bq_W = F
        applied_force.set_value([self.force_obj])


In [4]:
builder = DiagramBuilder()

time_step = 0.003
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step)
controller = builder.AddNamedSystem("controller",Controller())
Parser(plant, scene_graph).AddModels("../urdf_model/RR_planar.sdf")
plant.Finalize()
builder.Connect(controller.get_output_port(), plant.get_actuation_input_port())

In [5]:

end_effector_id = plant.GetBodyByName('end-effector').index()
force_sys = CartesianForceEE_Sys(end_effector_id)
builder.AddSystem(force_sys)
builder.Connect(force_sys.get_output_port(0), plant.get_applied_spatial_force_input_port())

In [6]:
visualizer = MeshcatVisualizer.AddToBuilder(
    builder, 
    scene_graph.get_query_output_port(),
    meshcat
)


In [7]:

diagram = builder.Build()
diagram_root_context = diagram.CreateDefaultContext()
    

In [8]:
plant_context = plant.GetMyContextFromRoot(diagram_root_context)

plant.GetJointByName('q1').set_angle(plant_context, -np.pi)
plant.GetJointByName('q2').set_angle(plant_context, np.pi/2)

#plant.get_actuation_input_port().FixValue(plant_context, np.ones(2))
#print(plant_context)
diagram.ForcedPublish(diagram_root_context)


In [9]:
simulator = Simulator(diagram, diagram_root_context)
simulator.AdvanceTo(50)
print(plant_context)

::_::plant Context
-------------------
Time: 50
States:
  1 discrete state groups with
     17 states
         2.190176894887891   2.590895334582189                   1                   0                   0                   0    1251.12500200004                   0  -12263.72626961825 -0.0273203942332893 -0.0157941673027195                   0                   0                   0   50.00100000000178                   0   -490.509809999895

Parameters:
  12 numeric parameter groups with
     1 parameters
       0
     1 parameters
       1
     1 parameters
       0
     1 parameters
       1
     10 parameters
       nan nan nan nan nan nan nan nan nan nan
     10 parameters
       1 0 0 0 1 1 1 0 0 0
     10 parameters
       1 0 0 0 1 1 1 0 0 0
     10 parameters
       1 0 0 0 1 1 1 0 0 0
     12 parameters
       1 0 0 0 1 0 0 0 1 0 0 0
     12 parameters
       1 0 0 0 1 0 0 0 1 0 0 0
     12 parameters
         1   0   0   0   1   0   0   0   1 0.5   0   0
     12 parameter

In [10]:
def save_plant_svg_graph(diagram, destination_path):
    dot_format_graph = graph_from_dot_data(diagram.GetGraphvizString())[0]
    binary_data_svg = dot_format_graph.create_svg()
    with open(destination_path, 'wb') as f:
        f.write(binary_data_svg)

save_plant_svg_graph(diagram, destination_path="../log/RR_planar.svg")